In [56]:
from bs4 import BeautifulSoup, NavigableString
import csv
import pandas as pd

In [57]:
# Record titles
html_article = '/Users/richardpang/Desktop/extract_maintext/extract_maintext/JM dataset/JM2018-2020.html'
titles_abstract = []
with open(html_article, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
titles=soup.find_all('h1',{'class':"delivery-title"})


for i in titles:
    title = i.get_text().replace(u'\xa0', u'')
    # Get abstract
    abstract = i.findNext('p').get_text()
    titles_abstract.append([title,abstract])

    
# Read articles
records = soup.find(id = "records")

# Delete keywords
for p in records.find_all('p', {"class":"body-paragraph"}):
    if p.get_text()[:9] == 'Keywords:':
        p.string = ''
    if p.get_text()[:18] == 'Online Supplement:':
        p.string = ''

# Delete tables
for table in records.find_all('table'):
    i = table.extract()
txt = records.get_text()



In [58]:
len('Online Supplement:')

18

In [59]:
len(titles_abstract)

57

# Citation scraping

In [60]:
import csv
import re
import time
from time import sleep
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [61]:
def make_url(keyword, conf, author, year, paper_id=None):
    """make url for search papers
    normal search (keyword, conf, author, year) or target search (paper_id)
    :param keyword: str or None
    :param conf: str or None, conference information
    :param author: str or None, author information
    :param year: int or None, published year
    :param paper_id: None or int, paper information
    :return: url
    """
    assert (
        keyword is not None
        or conf is not None
        or author is not None
        or year is not None
        or paper_id is not None
    ), "KeywordNotFoundError"
    url = "https://scholar.google.co.jp/scholar?"
    if paper_id is not None:
        url += f"&cites={paper_id}"
    else:
        url += "&as_sdt=0%2C5"
        if keyword is not None:
            url += f"&as_q={'%20'.join(keyword.split())}"
        else:
            url += "&as_q="
        if conf is not None:
            url += f"&as_publication={'%20'.join(conf.split())}"
        if author is not None:
            author = "+".join(author.split())
            url += f"&as_sauthors={'%20'.join(author.split())}"
        if year is not None:
            url += f"&as_ylo={year}"
    return url

In [62]:
def get_citations(soup):
    """obtain number of citations from soup
    :param soup: parsed html by BeautifulSoup
    :return: ci_num_list
    """
    tags3 = soup.find_all(text=re.compile("被引用"))
    ci_num_list = []
    for tag3 in tags3:
        # 被引用数取得
        citation = tag3.replace("被引用", "")[1:3]
        ci_num_list.append(int(citation))
    return ci_num_list

In [63]:
cite_data = []
for title in titles_lst:
    keyword = title
    url = make_url(keyword=keyword, conf=None, author=None, year=None)
    html_doc = requests.get(url).text
    soup = BeautifulSoup(html_doc, "html.parser")
    # print(get_citations(soup)
    cite_data.append([title,get_citations(soup)[0],len(get_citations(soup))]) 
    sleep(np.random.randint(5, 10))
df = pd.DataFrame(cite_data)
df.to_csv('/Users/richardpang/Desktop/extract_maintext/extract_maintext/citation_data.csv')


KeyboardInterrupt: 

# Data Processing

In [64]:
new_txt = txt.replace('Database:\n\t\t\t\t\t\tBusiness Source Complete', "partitionmark")
new_txt = new_txt.replace('Footnotes', "partitionmark")
new_txt = new_txt.replace('Supplemental Material', "partitionmark")
partition = new_txt.split('partitionmark')
# print("partition", len(partition))


In [65]:
len(partition[1])

70605

In [68]:
f = open('articles50.csv', 'w')
writer = csv.writer(f)
header = ['Number', "Title", 'Abstract','Body' ]
writer.writerow(header)
count = 1
for i in range (len(partition)):
    # print(i,len(partition[i]))
    text = partition[i]

    if len(text) > 20000:
        for pair in titles_abstract:
            title = pair[0]
            abstract = pair[1]
            if text[:50] in title:
                text = text[len(title)+1:]
                text = text[len(abstract):]


                row = [count,title,abstract,text]
                writer.writerow(row)
                count += 1
        # print(text)
        # if text[0].isupper():
        # print(count,partition[i][:30])
        # partition[i] is the main text
        # abstract = text.split('KEYWORDS_SPLIT',1)[0]
        # body = text.split('KEYWORDS_SPLIT',1)[1]
        # for title in titles_lst:
        #     if abstract[:50] in title:
        #         # print(title,abstract[:50])
        #         # print(abstract[:50])
        #         abstract = abstract[len(title):]
        #         # print(abstract)
        #         row = [count, title, abstract, body]
        #         writer.writerow(row)
        #         count += 1
print(count-1)

44


In [71]:
df = pd.read_csv('articles50.csv')
df

,Number,Title,Abstract,Body
0,1,(When) Does Third-Party Recognition for Design...,Third-party recognition for design excellence ...,Product design is a critical aspect of the inn...
1,2,A Cinderella Story: How Past Identity Salience...,"Like Cinderella, many repurposed products invo...",A worn-out airbag becomes a backpack. A leaky ...
2,3,A Commentary on “Transformative Marketing: The...,Transformative: Causing a marked change in som...,As a complement to the transformative marketin...
3,4,A Meta-Analysis of When and How Advertising Cr...,Although creativity is often considered a key ...,Creativity is important in marketing and is of...
4,5,A Study of Bidding Behavior in Voluntary-Pay P...,The authors investigate compliance behavior an...,"In a voluntary-pay auction format, all bidders..."
5,6,A Theories-in-Use Approach to Building Marketi...,This article's objective is to inspire and pro...,The marketing discipline is at a crossroads ([...
6,7,"A Theory of Customer Valuation: Concepts, Metr...",Customer value refers to the economic value of...,Any sustainable business first creates value f...
7,8,An Empirical Analysis of the Joint Effects of ...,This article develops a decision-making framew...,"In mid-2014, Coca-Cola launched Cola-Cola Life..."
8,9,An Integrated Power and Efficiency Model of Co...,"Power theories (e.g., social exchange theory, ...",The design of contractual governance between i...
9,10,Branding Cultural Products in International Ma...,Cultural products are a major component of the...,Cultural products are goods and services that ...


In [ ]:
import pandas as pd

df = pd.read_csv('/Users/richardpang/Desktop/extract_maintext/extract_maintext/articles50.csv')
count = 0
abstract_lst = []
for index, row in df.iterrows():
    number = row['Number']
    text = row['Body']
    count += 1
    abstract = text.split('KEYWORDS_SPLIT',1)[0]
    abstract_lst.append(abstract)

for abstract in abstract_lst:
    # print('ABSTRACT:', abstract)
    for title in titles_lst:
        # print("TITLE",title)
        small_title = abstract[:50]
        if small_title in title:
            print('abstract[:50]',small_title)
            # abstract = [title,abstract]
            print(True)

abstract_lst = pd.DataFrame(abstract_lst)
abstract_lst.to_csv('abstract.csv')
    # main_text = text.split('KEYWORDS_SPLIT',1)[1]
    # print(main_text)
    # print('Count:',count,'\n')

# print([title for title in titles_lst])

In [ ]:
for i in titles_lst:
    print(i)

A New Livestream Retail Analytics Framework to Assess the Sales Impact of Emotional Displays
Acknowledgments
An Emerging Theory of Avatar Marketing
Analyzing the Cultural Contradictions of Authenticity: Theoretical and Managerial Insights from the Market Logic of Conscious Capitalism
Artificial Intelligence in Utilitarian vs. Hedonic Contexts: The "Word-of-Machine" Effect
Augmented Reality in Retail and Its Impact on Sales
Befriending the Enemy: The Effects of Observing Brand-to-Brand Praise on Consumer Evaluations and Choices
Blame the Bot: Anthropomorphism and Anger in Customer–Chatbot Interactions
Can Encroachment Benefit Hotel Franchisees?
Carbon Footprinting and Pricing Under Climate Concerns
Communication in the Gig Economy: Buying and Selling in Online Freelance Marketplaces
Conducting Research in Marketing with Quasi-Experiments
Connecting to Place, People, and Past: How Products Make Us Feel Grounded
Despite Efficiencies, Mergers and Acquisitions Reduce Firm Value by Hurting C